In [3]:
import json
import pickle
import re
import os
from collections import defaultdict
from konlpy.tag import Mecab

In [4]:
mecab = Mecab()

In [5]:
output_dir = '/opt/ml/output/lstm-top12-weighted'

In [6]:
with open(os.path.join(output_dir, 'nbest_predictions.json'), 'r') as f:
    nbest = json.load(f)

In [7]:
with open('/opt/ml/input/data/test_dataset/sparse_bm25_data.bin', 'rb') as f:
    topk_dataset = pickle.load(f)

In [8]:
topk_dataset

,question,id,context,score
0,유령'은 어느 행성에서 지구로 왔는가?,mrc-1-000653-0,더크 젠틀리의 성스러운 탐정사무소의 줄거리는 이야기의 중추적인 부위에 자리잡은 시간...,21.768619
1,유령'은 어느 행성에서 지구로 왔는가?,mrc-1-000653-1,별하늘계의 변경에 위치한 외계행성 사만(サマーン) 출신의 여자아이. 지구 나이로는 ...,21.466410
2,유령'은 어느 행성에서 지구로 왔는가?,mrc-1-000653-2,HAT-P-2 b는 HATNet 프로젝트에 의해 2007년 5월 발견된 외계 행성이...,21.006212
3,유령'은 어느 행성에서 지구로 왔는가?,mrc-1-000653-3,페가수스자리 51 b가 발견되기 이전 폴란드 천문학자 알렉산데르 볼시찬이 이미 펄서...,19.322218
4,유령'은 어느 행성에서 지구로 왔는가?,mrc-1-000653-4,이 항성의 시선속도는 매우 일정해서 천문학자이자 행성 사냥꾼 제프리 마시는 랄랑드 ...,19.174997
...,...,...,...,...
3086,제2캐나다기갑여단이 상륙한 곳은?,mrc-0-003436-0,주노 해변 주노 해변은 쿠르쇨르메르의 양쪽으로 뻗은 5마일 정도의 해변이었다. 제3...,41.423084
3087,제2캐나다기갑여단이 상륙한 곳은?,mrc-0-003436-1,제3사단은 치명적인 운명을 맞은 영국 해외원정군의 일부였다. 이들은 제2차 세계 대...,34.467493
3088,제2캐나다기갑여단이 상륙한 곳은?,mrc-0-003436-2,캐나다군은 독일군의 베리에르 능선 방어가 여전히 철저하다는 것을 알고 있었다.D'E...,28.887549
3089,제2캐나다기갑여단이 상륙한 곳은?,mrc-0-003436-3,제2차 세계 대전 당시 불완전한 편제를 바탕으로 로저 에반스는 제1기갑사단을 이끌게...,28.636980


In [9]:
candidates = defaultdict(list)

for i in range(len(topk_dataset)):
    id = topk_dataset.iloc[i]['id']
    retriever_score = topk_dataset.iloc[i]['score']
    k = id.split('-')[-1]
    orig_id = '-'.join(id.split('-')[:-1])

    topk_preds = nbest[id]  # list of dict
    weighted_preds = [(p['text'], p['probability']*retriever_score) for p in topk_preds]
    candidates[orig_id].extend(weighted_preds)

In [10]:
blacklist = ['최소', '대략', '한때', '간혹', '최소한', '필수']

def valid_answer(text):
    # This function only tests for answers with Korean
    if not re.search('[가-힣]', text):
        return True
    elif text.strip() in blacklist:
        return False
        
    part_of_speech = [x[1][0] for x in mecab.pos(text)]

    # TODO: M tag 수식언만 나오는 경우? 
    if len(part_of_speech) == 1 and 'M' in part_of_speech:
        return False
    return True
    # return 'N' in part_of_speech

In [11]:
test_text = '무조건'

valid_answer(test_text)

False

In [12]:
predictions = {}
pattern = re.compile('[.\",\{\}\[\]\(\)\<\>]')

for id in candidates.keys():
    sorted_scores = sorted(candidates[id], key=lambda x:x[1], reverse=True)
    
    selected = 0
    # 정답이 next highest score 로 넘어가는 경우:
    # 1. 괄호, 부호 등을 제외하면 빈 string 일 때 
    # 2. 정답의 길이가 30자를 넘어갈 때
    # 3. 정답에 체언을 포함하지 않을 때
    while (re.sub(pattern, '', sorted_scores[selected][0]) == '' or len(sorted_scores[selected][0]) > 30 or not valid_answer(sorted_scores[selected][0])) and selected < len(sorted_scores)-1:
        selected += 1
    predictions[id] = sorted_scores[selected][0]
    

In [13]:
with open(os.path.join(output_dir, 'filtered_weighted_predictions.json'), 'w') as f:
    json.dump(predictions, f, ensure_ascii=False, indent = 4)

In [2]:
with open('/opt/ml/input/data/train_dataset/sparse_bm25_data.bin', 'rb') as f:
    topk_train_dataset = pickle.load(f)

In [3]:
topk_train_dataset

,question,id,context_id,context,contexts,context_ids,scores,original_context,answers
0,처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?,mrc-0-003264,10433,개인 또는 집단이 의사 결정 과정을 다른 사람에게 위임할 때 대리인 관계가 성립된다...,[개인 또는 집단이 의사 결정 과정을 다른 사람에게 위임할 때 대리인 관계가 성립된...,"[15066, 9027, 18014, 46019, 46589, 24584, 3624...","[22.235510912340906, 21.089564898131943, 18.32...","순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법...","{'answer_start': [284], 'text': ['한보철강']}"
1,스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는?,mrc-0-004762,46710,요크 카운티 동쪽에 처음으로 여객 열차 운행이 시작한 시점은 1868년 토론토 & ...,[요크 카운티 동쪽에 처음으로 여객 열차 운행이 시작한 시점은 1868년 토론토 &...,"[51765, 21916, 23373, 51850, 23376, 23375, 520...",[69.08883928533328],요크 카운티 동쪽에 처음으로 여객 열차 운행이 시작한 시점은 1868년 토론토 & ...,"{'answer_start': [146], 'text': ['1871년']}"
2,촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된 것은?,mrc-1-001810,11057,"촐라 정부 촐라의 정부 체제는 전제군주제였으며,2001 촐라의 군주는 절대적인 권력...","[촐라 정부 촐라의 정부 체제는 전제군주제였으며,2001 촐라의 군주는 절대적인 권...","[15694, 5300, 28167, 19132, 54890, 21545, 1141...","[23.506944591988223, 19.149940273916915]","촐라 정부 촐라의 정부 체제는 전제군주제였으며,2001 촐라의 군주는 절대적인 권력...","{'answer_start': [517], 'text': ['나뭇잎']}"
3,로타이르가 백조를 구하기 위해 사용한 것은?,mrc-1-000219,54365,프랑스의 십자군 무훈시는 1099년 예루살렘 왕국의 통치자가 된 고드프루아 드 부용...,[프랑스의 십자군 무훈시는 1099년 예루살렘 왕국의 통치자가 된 고드프루아 드 부...,"[59536, 59537, 59533, 59534, 59527, 59535, 653...",[35.59687889856637],프랑스의 십자군 무훈시는 1099년 예루살렘 왕국의 통치자가 된 고드프루아 드 부용...,"{'answer_start': [1109], 'text': ['금대야']}"
4,의견을 자유롭게 나누는 것은 조직 내 어떤 관계에서 가능한가?,mrc-1-000285,28216,탈관료제화는 현대사회에서 관료제 성격이 약화되는 현상이다. 현대사회에서 관료제는 약...,[탈관료제화는 현대사회에서 관료제 성격이 약화되는 현상이다. 현대사회에서 관료제는 ...,"[32991, 24309, 46612, 13391, 60258, 48589, 466...","[18.367841700728544, 18.24657620780058, 16.574...",탈관료제화는 현대사회에서 관료제 성격이 약화되는 현상이다. 현대사회에서 관료제는 약...,"{'answer_start': [386], 'text': ['수평적 관계']}"
...,...,...,...,...,...,...,...,...,...
235,전단이 연나라와의 전쟁에서 승리했을 당시 제나라의 왕은 누구인가?,mrc-0-000484,48200,기원전 284년에 이르러 당시 제나라의 왕이었던 제 민왕은 강력한 국력을 믿고 교만...,[기원전 284년에 이르러 당시 제나라의 왕이었던 제 민왕은 강력한 국력을 믿고 교...,"[53263, 53265, 53267, 53270, 52028, 53268, 532...","[39.95455292901735, 35.573960417694764, 30.360...","연나라 군대의 사령관이 악의에서 기겁으로 교체되자, 전단은 스스로 신령의 계시를 받...","{'answer_start': [1084], 'text': ['제 양왕']}"
236,공놀이 경기장 중 일부는 어디에 위치하고 있나?,mrc-0-002095,45293,경기장은 인사르 강 동쪽 둑에 있는 사란스크의 중심부에 위치해 있다. 경기장은 승용...,[경기장은 인사르 강 동쪽 둑에 있는 사란스크의 중심부에 위치해 있다. 경기장은 승...,"[50336, 47736, 12190, 18830, 18835, 50334, 778...","[21.168822242845998, 20.995204545313157, 20.76...",현재 우리가 볼 수 있는 티칼의 모습은 펜실베이니아 대학교와 과테말라 정부의 협조 ...,"{'answer_start': [343], 'text': [''일곱 개의 신전 광장..."
237,창씨개명령의 시행일을 미루는 것을 수락한 인물은?,mrc-0-003083,768,1940년 5월 1일 오전 창씨개명에 비협조적이라는 이유로 조선총독부 경무국에서 소...,[1940년 5월 1일 오전 창씨개명에 비협조적이라는 이유로 조선총독부 경무국에서 ...,"[5326, 34671, 7434, 9029, 6921, 11620, 6956, 8...",[25.326214908636324],1940년 5월 1일 오전 창씨개명에 비협조적이라는 이유로 조선총독부 경무국에서 소...,"{'answer_start': [247], 'text': ['미나미 지로']}"
238,망코 잉카가 쿠스코를 되찾기 위해 마련한 군사는 총 몇 명인가?,mrc-0-002978,43553,빌카밤바 지역은 파차쿠티 황제 때 부터 잉카 제국에 속해있던 지역이었다. 스페인 군...,[빌카밤바 지역은 파차쿠티 황제 때 부터 잉카 제국에 속해있던 지역이었다. 스페인 ...,"[48575, 55432, 55430, 56329, 48576, 55431, 265...","[47.44886519366105, 44.566859383571426, 42.593...",빌카밤바 지역은 파차쿠티 황제 때 부터 잉카 제국에 속해있던 지역이었다. 스페인 군...,"{'answer_start': [563], 'text': ['200,000명']}"


In [5]:
prev_k = 3

good = []  # minimal doc retrieval due to dynamic topk
bad = []    # missed doc retrieval due to dynamic topk

for i in range(len(topk_train_dataset)):
    ground_truth = topk_train_dataset.iloc[i]['original_context']
    top_n = len(topk_train_dataset.iloc[i]['scores'])
    contexts = topk_train_dataset.iloc[i]['contexts']

    if top_n < prev_k and ground_truth in contexts[:top_n]:
        good.append(i)
    elif top_n < prev_k and ground_truth in contexts[:prev_k]:
        bad.append(i)
    # elif top_n >= prev_k and ground_truth not in contexts[:len(top_n)]:
    #     bad.append(i)
print(len(good), len(bad))

140 0
